In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:
#loading the data
train=pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


****DATA PREPROCESSING****

In [3]:
# Check unique values in the 'job' feature

# Get the number of unique values
num_unique_jobs = train['job'].nunique()

# Get the actual unique values
unique_jobs = train['job'].unique()

# Print results
print("Number of unique job values:", num_unique_jobs)
print("Unique job values:", unique_jobs)


Number of unique job values: 12
Unique job values: ['technician' 'blue-collar' 'student' 'admin.' 'management' 'entrepreneur'
 'self-employed' 'unknown' 'services' 'retired' 'housemaid' 'unemployed']


In [4]:
# Count the frequency of each job category
job_counts = train['job'].value_counts()

print(job_counts)


job
management       175541
blue-collar      170498
technician       138107
admin.            81492
services          64209
retired           35185
self-employed     19020
entrepreneur      17718
unemployed        17634
housemaid         15912
student           11767
unknown            2917
Name: count, dtype: int64


In [5]:
# Group data by 'job' and calculate the mean of the target 'y'
# This tells us the probability of subscribing (y=1) for each job type
job_target_mean = train.groupby('job')['y'].mean()

# Sort the job types by the probability of subscription in descending order
job_target_sorted = job_target_mean.sort_values(ascending=False)

# Print the result
print("Jobs sorted by likelihood to subscribe to a bank term deposit:")
print(job_target_sorted)


Jobs sorted by likelihood to subscribe to a bank term deposit:
job
student          0.340784
retired          0.246241
unemployed       0.179823
management       0.150392
self-employed    0.129443
unknown          0.120672
technician       0.118321
admin.           0.116453
housemaid        0.084653
services         0.082714
entrepreneur     0.081386
blue-collar      0.067438
Name: y, dtype: float64


In [6]:
# ----------------------------
# TARGET ENCODING FOR 'job'
# ----------------------------

# Step 1: Map the sorted subscription probabilities to create 'job_encoded' in training set
train['job_encoded'] = train['job'].map(job_target_sorted)

# Step 2: Apply the same mapping to the test set
# Always use the mapping from the training set to avoid data leakage
test['job_encoded'] = test['job'].map(job_target_sorted)

# Step 3: Check the result
# Display first 10 rows of original job, encoded value, and target
print(train[['job', 'job_encoded', 'y']].head(10))


           job  job_encoded  y
0   technician     0.118321  0
1  blue-collar     0.067438  0
2  blue-collar     0.067438  0
3      student     0.340784  0
4   technician     0.118321  1
5       admin.     0.116453  0
6  blue-collar     0.067438  0
7       admin.     0.116453  0
8  blue-collar     0.067438  0
9   management     0.150392  0


In [7]:
'''
# Step 1: Calculate subscription probability per job
job_target_mean = train.groupby('job')['y'].mean()

# Step 2: Sort jobs by probability in ascending order (less likely = 0, most likely = highest number)
job_sorted = job_target_mean.sort_values().index  # returns job names sorted by subscription probability

# Step 3: Create a mapping from job name to integer label
job_label_mapping = {job: idx for idx, job in enumerate(job_sorted)}

# Step 4: Apply this mapping to train and test sets
train['job_encoded'] = train['job'].map(job_label_mapping)
test['job_encoded'] = test['job'].map(job_label_mapping)

# Step 5: Check result
print(train[['job', 'job_encoded', 'y']].head(10))




output:

           job  job_encoded  y
0   technician            5  0
1  blue-collar            0  0
2  blue-collar            0  0
3      student           11  0
4   technician            5  1
5       admin.            4  0
6  blue-collar            0  0
7       admin.            4  0
8  blue-collar            0  0
9   management            8  0
'''

"\n# Step 1: Calculate subscription probability per job\njob_target_mean = train.groupby('job')['y'].mean()\n\n# Step 2: Sort jobs by probability in ascending order (less likely = 0, most likely = highest number)\njob_sorted = job_target_mean.sort_values().index  # returns job names sorted by subscription probability\n\n# Step 3: Create a mapping from job name to integer label\njob_label_mapping = {job: idx for idx, job in enumerate(job_sorted)}\n\n# Step 4: Apply this mapping to train and test sets\ntrain['job_encoded'] = train['job'].map(job_label_mapping)\ntest['job_encoded'] = test['job'].map(job_label_mapping)\n\n# Step 5: Check result\nprint(train[['job', 'job_encoded', 'y']].head(10))\n\n\n\n\noutput:\n\n           job  job_encoded  y\n0   technician            5  0\n1  blue-collar            0  0\n2  blue-collar            0  0\n3      student           11  0\n4   technician            5  1\n5       admin.            4  0\n6  blue-collar            0  0\n7       admin.         

*in target encoding = Captures subtle differences between categories (e.g., 0.118321 vs 0.116453 is meaningful).*

*in lable encoding = blue-collar = 0.067 vs entrepreneur = 0.081 → both get integers 0 and 1, but the real difference is very small.*

*so we choose target encoding *

In [8]:
# Get unique values in 'marital' column
marital_unique = train['marital'].unique()
print("Unique marital values:", marital_unique)

# Get the number of unique values
marital_count = train['marital'].nunique()
print("Number of unique marital values:", marital_count)


Unique marital values: ['married' 'single' 'divorced']
Number of unique marital values: 3


In [9]:
# ----------------------------
# TARGET ENCODING FOR 'marital'
# ----------------------------

# Step 1: Calculate mean subscription probability for each marital status
marital_target_mean = train.groupby('marital')['y'].mean()

# Step 2: Sort marital statuses by probability of subscribing (optional)
marital_target_sorted = marital_target_mean.sort_values(ascending=False)

# Step 3: Map the probabilities to the training set
train['marital_encoded'] = train['marital'].map(marital_target_sorted)

# Step 4: Apply the same mapping to the test set
test['marital_encoded'] = test['marital'].map(marital_target_sorted)

# Step 5: Check the result
print(train[['marital', 'marital_encoded', 'y']].head(10))


    marital  marital_encoded  y
0   married         0.101872  0
1   married         0.101872  0
2   married         0.101872  0
3    single         0.170453  0
4   married         0.101872  1
5    single         0.170453  0
6   married         0.101872  0
7    single         0.170453  0
8   married         0.101872  0
9  divorced         0.111576  0


In [10]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_encoded,marital_encoded
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0,0.118321,0.101872
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0,0.067438,0.101872
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0,0.067438,0.101872
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0,0.340784,0.170453
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1,0.118321,0.101872


In [11]:
# Drop original categorical columns after encoding
train = train.drop(columns=['job', 'marital'])
test = test.drop(columns=['job', 'marital'])

# Check the first few rows
print(train.head())


   id  age  education default  balance housing loan   contact  day month  \
0   0   42  secondary      no        7      no   no  cellular   25   aug   
1   1   38  secondary      no      514      no   no   unknown   18   jun   
2   2   36  secondary      no      602     yes   no   unknown   14   may   
3   3   27  secondary      no       34     yes   no   unknown   28   may   
4   4   26  secondary      no      889     yes   no  cellular    3   feb   

   duration  campaign  pdays  previous poutcome  y  job_encoded  \
0       117         3     -1         0  unknown  0     0.118321   
1       185         1     -1         0  unknown  0     0.067438   
2       111         2     -1         0  unknown  0     0.067438   
3        10         2     -1         0  unknown  0     0.340784   
4       902         1     -1         0  unknown  1     0.118321   

   marital_encoded  
0         0.101872  
1         0.101872  
2         0.101872  
3         0.170453  
4         0.101872  


In [12]:
'''how does this works 
   # Step 1: Calculate mean subscription probability for each marital status
   marital_target_mean = train.groupby('marital')['y'].mean()


Explanation:

train.groupby('marital')

This groups the dataset by the values in the marital column.

For example, all rows where marital = married are grouped together, all rows with marital = single are another group, etc.

['y']

We select only the target column y (which indicates whether a client subscribed: 1 = yes, 0 = no).

.mean()

Calculates the average of y for each marital group.

Since y is 0 or 1, the mean is essentially the probability of subscription for that marital status.

Example: if 100 married people are in the dataset and 15 subscribed (y=1), the mean is 15/100 = 0.15.

'''

"how does this works \n   # Step 1: Calculate mean subscription probability for each marital status\n   marital_target_mean = train.groupby('marital')['y'].mean()\n\n\nExplanation:\n\ntrain.groupby('marital')\n\nThis groups the dataset by the values in the marital column.\n\nFor example, all rows where marital = married are grouped together, all rows with marital = single are another group, etc.\n\n['y']\n\nWe select only the target column y (which indicates whether a client subscribed: 1 = yes, 0 = no).\n\n.mean()\n\nCalculates the average of y for each marital group.\n\nSince y is 0 or 1, the mean is essentially the probability of subscription for that marital status.\n\nExample: if 100 married people are in the dataset and 15 subscribed (y=1), the mean is 15/100 = 0.15.\n\n"

In [13]:
# Get unique values in 'education' column
education_unique = train['education'].unique()
print("Unique education values:", education_unique)

# Get the number of unique values
education_count = train['education'].nunique()
print("Number of unique education values:", education_count)

# Optionally, see the frequency of each education level
print(train['education'].value_counts())


Unique education values: ['secondary' 'primary' 'tertiary' 'unknown']
Number of unique education values: 4
education
secondary    401683
tertiary     227508
primary       99510
unknown       21299
Name: count, dtype: int64


In [14]:
# ----------------------------
# TARGET ENCODING FOR 'education'
# ----------------------------

# Step 1: Calculate mean subscription probability for each education level
education_target_mean = train.groupby('education')['y'].mean()

# Step 2: Sort education levels by probability of subscribing (optional)
education_target_sorted = education_target_mean.sort_values(ascending=False)

# Step 3: Map the probabilities to the training set
train['education_encoded'] = train['education'].map(education_target_sorted)

# Step 4: Apply the same mapping to the test set
test['education_encoded'] = test['education'].map(education_target_sorted)

# Step 5: Check the result
print(train[['education', 'education_encoded', 'y']].head(10))


   education  education_encoded  y
0  secondary           0.105491  0
1  secondary           0.105491  0
2  secondary           0.105491  0
3  secondary           0.105491  0
4  secondary           0.105491  1
5  secondary           0.105491  0
6  secondary           0.105491  0
7  secondary           0.105491  0
8    primary           0.083097  0
9   tertiary           0.162649  0


In [15]:
# Unique values in 'default' column
default_unique = train['default'].unique()
print("Unique default values:", default_unique)

# Number of unique values
default_count = train['default'].nunique()
print("Number of unique default values:", default_count)

# Frequency of each value
print(train['default'].value_counts())


Unique default values: ['no' 'yes']
Number of unique default values: 2
default
no     737151
yes     12849
Name: count, dtype: int64


In [16]:
# Simple label encoding for 'default'
train['default_encoded'] = train['default'].map({'no': 0, 'yes': 1})
test['default_encoded'] = test['default'].map({'no': 0, 'yes': 1})

# Optional: drop original column
train = train.drop(columns=['default'])
test = test.drop(columns=['default'])

# Check the result
print(train[['default_encoded', 'y']].head(10))


   default_encoded  y
0                0  0
1                0  0
2                0  0
3                0  0
4                0  1
5                0  0
6                0  0
7                0  0
8                0  0
9                0  0


In [17]:
# Drop the original 'education' column after encoding
train = train.drop(columns=['education'])
test = test.drop(columns=['education'])


In [18]:
# Number of unique values in 'balance'
balance_unique_count = train['balance'].nunique()
print("Number of unique balance values:", balance_unique_count)

# List of unique values (optional, might be very long)
balance_unique_values = train['balance'].unique()
#print("Unique balance values:", balance_unique_values)

# Number of missing/null values in 'balance'
balance_null_count = train['balance'].isnull().sum()
print("Number of null values in balance:", balance_null_count)


Number of unique balance values: 8217
Number of null values in balance: 0


In [19]:
# Unique values in 'housing'
housing_unique = train['housing'].unique()
print("Unique housing values:", housing_unique)

# Number of unique values
print("Number of unique housing values:", train['housing'].nunique())

# Frequency of each value
print(train['housing'].value_counts())


Unique housing values: ['no' 'yes']
Number of unique housing values: 2
housing
yes    411288
no     338712
Name: count, dtype: int64


In [20]:
# Simple label encoding for 'housing'
train['housing_encoded'] = train['housing'].map({'no': 0, 'yes': 1})
test['housing_encoded'] = test['housing'].map({'no': 0, 'yes': 1})

# Optional: drop original column
train = train.drop(columns=['housing'])
test = test.drop(columns=['housing'])

# Check result
print(train[['housing_encoded', 'y']].head(10))


   housing_encoded  y
0                0  0
1                0  0
2                1  0
3                1  0
4                1  1
5                1  0
6                0  0
7                0  0
8                0  0
9                1  0


In [21]:
# Unique values in 'loan'
loan_unique = train['loan'].unique()
print("Unique loan values:", loan_unique)

# Number of unique values
print("Number of unique loan values:", train['loan'].nunique())

# Frequency of each value
print(train['loan'].value_counts())


Unique loan values: ['no' 'yes']
Number of unique loan values: 2
loan
no     645023
yes    104977
Name: count, dtype: int64


In [22]:
# Label encoding for 'loan'
train['loan_encoded'] = train['loan'].map({'no': 0, 'yes': 1})
test['loan_encoded'] = test['loan'].map({'no': 0, 'yes': 1})

# Optional: drop original column
train = train.drop(columns=['loan'])
test = test.drop(columns=['loan'])

# Check first few rows
print(train[['loan_encoded', 'y']].head(10))


   loan_encoded  y
0             0  0
1             0  0
2             0  0
3             0  0
4             0  1
5             0  0
6             0  0
7             0  0
8             0  0
9             0  0


In [23]:
# Unique values in 'contact'
contact_unique = train['contact'].unique()
print("Unique contact values:", contact_unique)

# Number of unique values
print("Number of unique contact values:", train['contact'].nunique())

# Frequency of each value
print(train['contact'].value_counts())


Unique contact values: ['cellular' 'unknown' 'telephone']
Number of unique contact values: 3
contact
cellular     486655
unknown      231627
telephone     31718
Name: count, dtype: int64


In [24]:
# One-hot encode 'contact' column
train_contact_ohe = pd.get_dummies(train['contact'], prefix='contact')
test_contact_ohe = pd.get_dummies(test['contact'], prefix='contact')

# Align columns of train and test (in case some category is missing in test)
train_contact_ohe, test_contact_ohe = train_contact_ohe.align(test_contact_ohe, join='outer', axis=1, fill_value=0)

# Add one-hot columns to original dataset
train = pd.concat([train, train_contact_ohe], axis=1)
test = pd.concat([test, test_contact_ohe], axis=1)

# Optional: drop original 'contact' column
train = train.drop(columns=['contact'])
test = test.drop(columns=['contact'])

# Check first few rows
print(train.head(5))


   id  age  balance  day month  duration  campaign  pdays  previous poutcome  \
0   0   42        7   25   aug       117         3     -1         0  unknown   
1   1   38      514   18   jun       185         1     -1         0  unknown   
2   2   36      602   14   may       111         2     -1         0  unknown   
3   3   27       34   28   may        10         2     -1         0  unknown   
4   4   26      889    3   feb       902         1     -1         0  unknown   

   y  job_encoded  marital_encoded  education_encoded  default_encoded  \
0  0     0.118321         0.101872           0.105491                0   
1  0     0.067438         0.101872           0.105491                0   
2  0     0.067438         0.101872           0.105491                0   
3  0     0.340784         0.170453           0.105491                0   
4  1     0.118321         0.101872           0.105491                0   

   housing_encoded  loan_encoded  contact_cellular  contact_telephone  \
0

In [25]:
# Unique values in 'day'
day_unique = train['day'].unique()
print("Unique day values:", day_unique)

# Number of unique values
print("Number of unique day values:", train['day'].nunique())

# Frequency of each value
print(train['day'].value_counts())


Unique day values: [25 18 14 28  3 20 21 31  4  8  5 24 11 12 26 30  6 10 19 17  2 13 29 23
 27  7  9 16  1 22 15]
Number of unique day values: 31
day
20    51431
18    41352
21    36395
28    33479
17    33428
14    32854
8     30996
6     30573
5     30245
19    29948
29    29858
7     28771
15    27502
30    26181
12    25622
11    25239
9     24752
13    24488
4     22270
16    20909
2     20003
27    19586
26    16216
3     15827
22    15128
23    14539
25    13492
31    11159
10     7626
24     6241
1      3890
Name: count, dtype: int64


In [26]:
# Display first 10 rows of 'day' column
print(train['day'].head(10))

# Or display the entire column (careful if dataset is large)
print(train['day'])


0    25
1    18
2    14
3    28
4     3
5    20
6    21
7    31
8     4
9     8
Name: day, dtype: int64
0         25
1         18
2         14
3         28
4          3
          ..
749995     4
749996    19
749997    17
749998    26
749999     4
Name: day, Length: 750000, dtype: int64


In [27]:
# Total number of rows in the training set
total_rows = train.shape[0]
print("Total number of rows:", total_rows)


Total number of rows: 750000


In [28]:
# Unique values in 'month'
month_unique = train['month'].unique()
print("Unique month values:", month_unique)

# Number of unique month values
print("Number of unique month values:", train['month'].nunique())

# Frequency of each month
print(train['month'].value_counts())


Unique month values: ['aug' 'jun' 'may' 'feb' 'apr' 'nov' 'jul' 'jan' 'oct' 'mar' 'sep' 'dec']
Number of unique month values: 12
month
may    228411
aug    128859
jul    110647
jun     93670
nov     66062
apr     41319
feb     37611
jan     18937
oct      9204
sep      7409
mar      5802
dec      2069
Name: count, dtype: int64


What is Cyclical Encoding?

Some features are cyclical, meaning the first and last values are close to each other in meaning:

Months: Dec → Jan

Hours: 23 → 0

Days of the week: Sun → Mon

If you treat them as numeric (1–12 for months), the model might think Dec (12) is far from Jan (1), which is misleading.

Cyclical encoding fixes this using sine and cosine transformations:

In [29]:
# ----------------------------
# TARGET ENCODING FOR 'month'
# ----------------------------

# Step 1: Calculate mean subscription probability for each month
month_target_mean = train.groupby('month')['y'].mean()

# Step 2: Sort months by subscription probability (optional, just for checking)
month_target_sorted = month_target_mean.sort_values(ascending=False)
print("Months sorted by likelihood to subscribe:")
print(month_target_sorted)

# Step 3: Map the target mean to create 'month_encoded' in training set
train['month_encoded'] = train['month'].map(month_target_mean)

# Step 4: Apply the same mapping to the test set
# Always use the mapping from the training set to avoid data leakage
test['month_encoded'] = test['month'].map(month_target_mean)

# Step 5: Check the result
print(train[['month', 'month_encoded', 'y']].head(10))


Months sorted by likelihood to subscribe:
month
mar    0.571355
sep    0.534755
dec    0.513291
oct    0.490004
apr    0.235654
feb    0.206801
jan    0.124148
aug    0.112161
nov    0.109806
jun    0.103726
jul    0.090847
may    0.071354
Name: y, dtype: float64
  month  month_encoded  y
0   aug       0.112161  0
1   jun       0.103726  0
2   may       0.071354  0
3   may       0.071354  0
4   feb       0.206801  1
5   apr       0.235654  0
6   nov       0.109806  0
7   jul       0.090847  0
8   aug       0.112161  0
9   may       0.071354  0


In [30]:
# Check basic stats of 'duration'
print(train['duration'].describe())

# Check for unique values count
print("Number of unique duration values:", train['duration'].nunique())

# Optional: see top 10 values
print(train['duration'].head(10))


count    750000.000000
mean        256.229144
std         272.555662
min           1.000000
25%          91.000000
50%         133.000000
75%         361.000000
max        4918.000000
Name: duration, dtype: float64
Number of unique duration values: 1760
0     117
1     185
2     111
3      10
4     902
5    1010
6      90
7      49
8      50
9     119
Name: duration, dtype: int64


In [31]:
# Number of unique values in 'campaign'
print("Number of unique campaign values:", train['campaign'].nunique())

# Display the unique values themselves
print("Unique campaign values:", train['campaign'].unique())

# Optional: Get value counts to see frequency of each number
print("Value counts for 'campaign':")
print(train['campaign'].value_counts().sort_index())


Number of unique campaign values: 52
Unique campaign values: [ 3  1  2 25  5  7  4 11  6  9  8 16 10 15 12 13 19 17 14 22 23 34 18 29
 24 30 21 27 36 28 20 35 32 26 40 37 31 33 63 51 46 39 43 48 41 44 38 42
 55 50 58 45]
Value counts for 'campaign':
campaign
1     304481
2     209834
3      88683
4      60974
5      25648
6      19694
7      10360
8       7777
9       3993
10      3511
11      2576
12      2027
13      1720
14      1231
15      1099
16      1018
17       902
18       678
19       556
20       530
21       414
22       272
23       236
24       226
25       262
26       134
27       100
28       176
29       182
30       103
31       153
32        88
33        64
34        48
35        55
36        31
37        30
38        30
39         8
40        13
41        19
42         3
43        16
44        10
45         4
46         8
48         2
50        11
51         7
55         1
58         1
63         1
Name: count, dtype: int64


In [32]:
# Check basic stats of 'pdays'
print(train['pdays'].describe())

# Number of unique values
print("Number of unique pdays values:", train['pdays'].nunique())

# Unique values themselves (optional, might be a lot)
print("Unique pdays values (sample):", train['pdays'].unique()[:20])

# Value counts to see frequency of each value
print("Value counts for 'pdays':")
print(train['pdays'].value_counts().sort_index())


count    750000.000000
mean         22.412733
std          77.319998
min          -1.000000
25%          -1.000000
50%          -1.000000
75%          -1.000000
max         871.000000
Name: pdays, dtype: float64
Number of unique pdays values: 596
Unique pdays values (sample): [ -1 175  91 181 252  90 211 183  87 168 172 364 266 122  94 196 324  92
 348  97]
Value counts for 'pdays':
pdays
-1      672434
 0           1
 1          45
 2         210
 3           1
         ...  
 838         4
 842         5
 850         1
 854         2
 871         2
Name: count, Length: 596, dtype: int64


In [33]:
# Number of unique values in 'previous'
print("Number of unique values in 'previous':", train['previous'].nunique())

# Display the unique values themselves
print("Unique 'previous' values:", train['previous'].unique())

# Value counts to see how often each number occurs
print("Value counts for 'previous':")
print(train['previous'].value_counts().sort_index())


Number of unique values in 'previous': 50
Unique 'previous' values: [  0   3   4   2   1   5   6  10  11   9   7  14  13  17   8  24  23  37
  12  27  25  38  15  16  29  19  20  32  55  18  34  22  26  21  28  35
  39  51  31  43  30  36  33  41  40  47  46 200  48  58]
Value counts for 'previous':
previous
0      672431
1       28342
2       20468
3       10326
4        6239
5        3882
6        2183
7        1730
8        1036
9         697
10        501
11        503
12        320
13        327
14        137
15        130
16         81
17        103
18         56
19         77
20         56
21         22
22         43
23         74
24         34
25         20
26         17
27         29
28         23
29         29
30         15
31          9
32          5
33          3
34          3
35          6
36          8
37          7
38          6
39          2
40          1
41          4
43          4
46          1
47          1
48          1
51          4
55          2
58          1
200 

In [34]:
# Check unique values in 'poutcome'
print("Unique 'poutcome' values:", train['poutcome'].unique())

# Number of unique values
print("Number of unique 'poutcome' values:", train['poutcome'].nunique())

# Value counts to see distribution
print("Value counts for 'poutcome':")
print(train['poutcome'].value_counts())


Unique 'poutcome' values: ['unknown' 'other' 'failure' 'success']
Number of unique 'poutcome' values: 4
Value counts for 'poutcome':
poutcome
unknown    672450
failure     45115
success     17691
other       14744
Name: count, dtype: int64


In [35]:
# Perform one-hot encoding for 'poutcome' in the training set
poutcome_dummies_train = pd.get_dummies(train['poutcome'], prefix='poutcome')

# Concatenate the new one-hot columns to the original training dataframe
train = pd.concat([train, poutcome_dummies_train], axis=1)

# Do the same for the test set
poutcome_dummies_test = pd.get_dummies(test['poutcome'], prefix='poutcome')
test = pd.concat([test, poutcome_dummies_test], axis=1)

# Optional: Drop the original 'poutcome' column if you don't need it anymore
train.drop('poutcome', axis=1, inplace=True)
test.drop('poutcome', axis=1, inplace=True)

# Check the first few rows to see the one-hot encoded columns
print(train.head())


   id  age  balance  day month  duration  campaign  pdays  previous  y  ...  \
0   0   42        7   25   aug       117         3     -1         0  0  ...   
1   1   38      514   18   jun       185         1     -1         0  0  ...   
2   2   36      602   14   may       111         2     -1         0  0  ...   
3   3   27       34   28   may        10         2     -1         0  0  ...   
4   4   26      889    3   feb       902         1     -1         0  1  ...   

   housing_encoded  loan_encoded  contact_cellular  contact_telephone  \
0                0             0              True              False   
1                0             0             False              False   
2                1             0             False              False   
3                1             0             False              False   
4                1             0              True              False   

   contact_unknown  month_encoded  poutcome_failure  poutcome_other  \
0            Fa

In [36]:
train.head()

,id,age,balance,day,month,duration,campaign,pdays,previous,y,...,housing_encoded,loan_encoded,contact_cellular,contact_telephone,contact_unknown,month_encoded,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0,42,7,25,aug,117,3,-1,0,0,...,0,0,True,False,False,0.112161,False,False,False,True
1,1,38,514,18,jun,185,1,-1,0,0,...,0,0,False,False,True,0.103726,False,False,False,True
2,2,36,602,14,may,111,2,-1,0,0,...,1,0,False,False,True,0.071354,False,False,False,True
3,3,27,34,28,may,10,2,-1,0,0,...,1,0,False,False,True,0.071354,False,False,False,True
4,4,26,889,3,feb,902,1,-1,0,1,...,1,0,True,False,False,0.206801,False,False,False,True


In [37]:
# Columns in the training set
print("Training set columns:")
print(train.columns)

# Columns in the test set
print("\nTest set columns:")
print(test.columns)


Training set columns:
Index(['id', 'age', 'balance', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'y', 'job_encoded', 'marital_encoded', 'education_encoded',
       'default_encoded', 'housing_encoded', 'loan_encoded',
       'contact_cellular', 'contact_telephone', 'contact_unknown',
       'month_encoded', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown'],
      dtype='object')

Test set columns:
Index(['id', 'age', 'balance', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'job_encoded', 'marital_encoded', 'education_encoded',
       'default_encoded', 'housing_encoded', 'loan_encoded',
       'contact_cellular', 'contact_telephone', 'contact_unknown',
       'month_encoded', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown'],
      dtype='object')


In [38]:
# List all column names in the training dataset
print(train.columns)

Index(['id', 'age', 'balance', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'y', 'job_encoded', 'marital_encoded', 'education_encoded',
       'default_encoded', 'housing_encoded', 'loan_encoded',
       'contact_cellular', 'contact_telephone', 'contact_unknown',
       'month_encoded', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown'],
      dtype='object')


In [39]:
# Drop the original 'month' column since 'month_encoded' exists
train.drop(columns=['month'], inplace=True)
test.drop(columns=['month'], inplace=True)

# Check remaining columns
print(train.columns)


Index(['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous', 'y', 'job_encoded', 'marital_encoded', 'education_encoded',
       'default_encoded', 'housing_encoded', 'loan_encoded',
       'contact_cellular', 'contact_telephone', 'contact_unknown',
       'month_encoded', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown'],
      dtype='object')


In [40]:
# Separate features and target
X_train = train.drop(columns=['id', 'y'])
y_train = train['y']

# For test set, drop 'id' column only
X_test = test.drop(columns=['id'])


In [41]:
from sklearn.preprocessing import StandardScaler

# List of numeric columns to scale
numeric_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Initialize the scaler
scaler = StandardScaler()

# Fit on training data and transform
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

# Transform the test set using the same scaler
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Check first 5 rows after scaling
print(X_train.head())


        age   balance       day  duration  campaign     pdays  previous  \
0  0.106310 -0.422083  1.076594 -0.510829  0.155597 -0.302803 -0.223475   
1 -0.289776 -0.243316  0.228194 -0.261338 -0.580100 -0.302803 -0.223475   
2 -0.487819 -0.212287 -0.256606 -0.532843 -0.212251 -0.302803 -0.223475   
3 -1.379012 -0.412563  1.440194 -0.903409 -0.212251 -0.302803 -0.223475   
4 -1.478033 -0.111092 -1.589805  2.369319 -0.580100 -0.302803 -0.223475   

   job_encoded  marital_encoded  education_encoded  ...  housing_encoded  \
0     0.118321         0.101872           0.105491  ...                0   
1     0.067438         0.101872           0.105491  ...                0   
2     0.067438         0.101872           0.105491  ...                1   
3     0.340784         0.170453           0.105491  ...                1   
4     0.118321         0.101872           0.105491  ...                1   

   loan_encoded  contact_cellular  contact_telephone  contact_unknown  \
0             0    

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Step 1: Initialize the model
model = LogisticRegression(max_iter=1000, random_state=42)

# Step 2: Train the model on the training data
model.fit(X_train, y_train)

# Step 3: Predict probabilities on the training set (to check ROC-AUC)
y_train_pred_proba = model.predict_proba(X_train)[:, 1]

# Step 4: Calculate ROC-AUC score on training set
roc_auc = roc_auc_score(y_train, y_train_pred_proba)
print(f"Training ROC-AUC Score: {roc_auc:.4f}")

# Step 5: Predict probabilities on the test set for submission
y_test_pred_proba = model.predict_proba(X_test)[:, 1]

# Step 6: Create submission DataFrame
submission = pd.DataFrame({
    'id': test['id'],
    'y': y_test_pred_proba
})

# Step 7: Save submission file
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")


Training ROC-AUC Score: 0.9399
Submission file created: submission.csv
